In [1]:
import spacy
spacy.load('en')
from spacy.lang.en import English

import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

from gensim import corpora
import pandas as pd
import random
import pickle

import warnings
warnings.filterwarnings('ignore')

C:\Users\Gab\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\Gab\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
C:\Users\Gab\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\Gab\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
C:\Users\Gab\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warni

In [2]:
# Tokenize
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [3]:
nltk.download('wordnet')

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
fil_stop = []
with open('assets/stop_words_ph.txt') as f:
    fil_stop.append(f.readlines())    

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Filters here
unwanted = ['like', 'know']
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in unwanted]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [token for token in tokens if token not in fil_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [6]:
text_data = []
df = pd.read_csv('data/posts_data.csv')
data = df['Body']
data.dropna(inplace=True)

In [7]:
for line in data:
    tokens = prepare_text_for_lda(line)
    text_data.append(tokens)
    print('Original Sentence:', line)
    print('Tokens:', tokens)
    print()

Original Sentence: Does anyone get blisters on their feet from walking so much around school?
Tokens: ['anyone', 'blister', 'foot', 'walking', 'much', 'around', 'school']

Original Sentence: It's been a few months since that whole sexual harassment scandal rocked ateneo. The silence is deafening. Some of the profs who have been so vocal about issues like martial law etc never spoke up about this particular issue. And now they're opening up about that fucking upsilon scandal like it's the worst shit ever? Fuck you hypocrites.
Tokens: ['month', 'since', 'whole', 'sexual', 'harassment', 'scandal', 'rock', 'ateneo', 'silence', 'deafen', 'prof', 'vocal', 'issue', 'martial', 'never', 'spoke', 'particular', 'issue', 'opening', 'fucking', 'upsilon', 'scandal', 'worst', 'shit', 'ever', 'fuck', 'hypocrite']

Original Sentence: I think the reason why we really hate Filipino in college is not because of the language, but the topics in it. I’d understand if Filipino was about learning how to speak 

Original Sentence: I miss my S.O. back at home so bad... I've been restless for the past couple of weeks and I can't wait to go home. I've been super nervous and fidgety ever since she stopped replying to my messages often. I tell myself "She's busy, she's studying, she's with her friends, you gotta give her some space man." But no matter how hard I try, that annoying thought lingers in my mind that she isn't interested in me anymore. I can't even tell her how I feel because she rarely ever replies. And yet, I tell myself "She's busy with academics, give her space and it will all work out in the end... I hope :( "
Tokens: ['miss', 's.o.', 'back', 'home', 'restless', 'past', 'couple', 'week', 'wait', 'home', 'super', 'nervous', 'fidgety', 'ever', 'since', 'stop', 'reply', 'message', 'often', 'tell', 'busy', 'studying', 'friend', 'give', 'space', 'matter', 'hard', 'annoying', 'thought', 'linger', 'mind', 'interest', 'anymore', 'even', 'tell', 'feel', 'rarely', 'ever', 'reply', 'tell', 'b

We are not limited to the things that try to put us in a box. We all have our own form of beauty and intelligence and our lives are about using it to it's fullest potential. It's not about grades. It's not about what you get, but who you become.  So if you're reading this, and you're going through a tough time in these final weeks, remember that while you always do your best, never think that you have something to prove to anyone, that you're trying to validate something. Control what you can control. Existence precedes essence.
Tokens: ['believe', 'school', 'condition', 'perfectionist', 'excel', 'purpose', 'learning', 'improvement', 'fear', 'failure', 'inadequacy', 'embed', 'brain', 'smart', 'guy', 'reward', 'failure', 'graduate', 'student', 'robot', 'people', 'limited', 'things', 'form', 'beauty', 'intelligence', 'life', 'using', 'full', 'potential', 'grade', 'become', 'reading', 'going', 'tough', 'time', 'final', 'week', 'remember', 'always', 'best', 'never', 'think', 'something', '

Tokens: ['double', 'standard', 'sexism', 'prejudice', 'apply', 'woman', 'instead', 'believing', 'victim', 'grow', 'fuck', 'watch', 'shapiro', 'video', 'fockin', 'snowflake']

Original Sentence: I mean the react sticker system is cool and all, but its still confusing especially in certain news posts. Are the angry reacts because of being angry at the news or at the website for attacking a certain someone?
Tokens: ['mean', 'react', 'sticker', 'system', 'cool', 'still', 'confuse', 'especially', 'certain', 'news', 'post', 'angry', 'react', 'angry', 'news', 'website', 'attack', 'certain', 'someone']

Original Sentence: "When you try your best but you don't succeed.."

This line from Coldplay's Fix You keeps replaying in my head.. this is how I feel with Philo 101.. :(( I really do try my best to explain the topics in the exams but i still get a low grade :(( How to get a good grade in Philo?! Please help huhuhu
Tokens: ['best', 'succeed', 'line', 'coldplay', 'keep', 'replay', 'head', 'feel'

Yo that guard is heckin solid dude so if you have classes there or you're just passing by Faura, make sure you greet him cause he'll surely greet you with the usual "Hi Ma'am/Sir!" Just an appreciation post for all the guards arounds campus because they really do care about you! P.S. He gives solid advice and he has this super duper kuwentuhan buddy vibe wah pls protect this man
Tokens: ['friend', 'ask', 'best', 'person', 'first', 'year', 'college', 'blurt', 'kuya', 'resty', 'faura', 'guard', 'heckin', 'solid', 'dude', 'class', 'passing', 'faura', 'make', 'sure', 'greet', 'cause', 'surely', 'greet', 'usual', "ma'am", 'appreciation', 'post', 'guard', 'arounds', 'campus', 'really', 'care', 'p.s.', 'give', 'solid', 'advice', 'super', 'duper', 'kuwentuhan', 'buddy', 'vibe', 'protect']

Original Sentence: Hello "engejowa!!!" :) Are you still looking for a jowa?
Tokens: ['hello', 'engejowa', 'still', 'looking', 'jowa']

Original Sentence: Everybody has a story behind that cigarette that they

...

Tokens: ['guy', 'stand', 'issue', 'young', 'sinatra', 'carter', 'rattpack']

Original Sentence: I don't even try to get to know my blockmates

the less I know, the less disappointed I can be :-))))))) backstabbers suck
Tokens: ['even', 'blockmates', 'le', 'le', 'disappoint', 'backstabbers', 'suck']

Original Sentence: I get easily attracted to guys who are both smart and humble
Tokens: ['easily', 'attract', 'guy', 'smart', 'humble']

Original Sentence: hi am a freshie can i rely on what i remember for a midterm and yolo it or up to how much should i study ? asking for a friend
Tokens: ['freshie', 'rely', 'remember', 'midterm', 'yolo', 'much', 'study', 'asking', 'friend']

Original Sentence: Can we pls get this wrestling org shit trending? Idk many ppl who wanna talk abt it. Let's gather na!!!
Tokens: ['wrestling', 'shit', 'trend', 'many', 'wanna', 'talk', 'gather']

Original Sentence: wanna watch UAAP with me on october 6?? <3 moa arena mismoo
Tokens: ['wanna', 'watch', 'uaap', 'octobe

Tokens: ['kita', 'kita', 'leong', 'napaisip', 'nagtanong', 'iisa', 'lang', 'hindi', 'iyong', 'sinabi', 'joke', 'please', 'stay', 'bukas', 'labi', 'tamis', 'iyong', 'ngiti', 'napapansin', 'ayos', 'batid', 'agos', 'oras', 'ngayon', 'heto', 'naman', 'heto', 'naman', 'naiinlove', 'bumuhos', 'ulan', 'naalala', 'kanta', 'ryan', 'lunurin', 'nang', 'tuluyan', 'bukas', 'nang', 'bayong', 'stay', 'close', 'akbay', 'silong', 'sasabihan', 'kitang', 'mahal', 'kita', 'napapansin', 'ayos', 'batid', 'agos', 'oras', 'ngayon', 'heto', 'naman', 'heto', 'naman', 'naiinlove', 'napapansin', 'ayos', 'batid', 'agos', 'oras', 'ngayon', 'heto', 'naman', 'heto', 'naman', 'naiinlove']

Original Sentence: Kinakalabit ka na nga
‘Di ka pa rin namamansin
Busyng busy ka yata na tumingin ng iba
 Hetong payo ko sa ‘yo
 Maraming humahaligi
 Bigyan mo lang ng kaunting pansin ‘Di ka ba napapagod
 Ang daming nagpupuyat para lang
 Makasama ka kahit sandali
 Ilang pagsubok pa ba ang pagdaraanan
 Para lang mapatunayan na
 May s

~SSS, SABIHIN SA SANGGU~
Tokens: ['asfdhdjdkf', 'sabihin', 'sanggu', 'jingle', '~sss', 'sabihin', 'sanggu~']

Original Sentence: Petition to have an ADMU singles group/page school effort para mabawasan ang mga sawi!!!!!
Tokens: ['petition', 'admu', 'singles', 'group', 'page', 'school', 'effort', 'para', 'mabawasan', 'sawi']

Original Sentence: It is hard to be in a class with no friends because I have no one to turn to when I am already uncomfortable with the prof.
Tokens: ['hard', 'class', 'friend', 'turn', 'already', 'uncomfortable', 'prof']

Original Sentence: L A M P
Tokens: []

Original Sentence: Real talk.

I think the LGBTQ+ community is creating too many terms too fast. I don't want to be misunderstood but here ah. I get that you want to express yourself fully, that not every word used and their meanings can fully describe the complexity of your sexualities and genders but the charge to come up with more words shouldn't be a charge. It should be a slower, methodical, and well t

Hello. We are the hosts of the 1v1 tourney that was posted here a few days ago. Due to the large amount of students that signed up and its high demand, we will be hosting a DUOS tourney after the first tourney is finished, complete with CASH PRIZES for 1st, 2nd, and 3rd place (P250 entry fee per duo. More info soon). REGISTER: bit.ly/fnbrduosDN Only 32 players (16 duos) will be chosen so register yourself and your duo partner ASAP! Limited to PC, PS4, and XBOX players ONLY. P.S. For the time being, only students of ADMU (+AHS), UPD, Xavier, DLSU (+SHS), and CSB will be accommodated.
Tokens: ['fortnite', 'tournament', 'hello', 'host', 'tourney', 'post', 'days', 'large', 'amount', 'student', 'sign', 'high', 'demand', 'host', 'duo', 'tourney', 'first', 'tourney', 'finish', 'complete', 'cash', 'prize', 'place', 'p250', 'entry', 'info', 'soon', 'register', 'bit.ly/fnbrduosdn', 'player', 'duo', 'chosen', 'register', 'partner', 'asap', 'limited', 'xbox', 'player', 'p.s.', 'time', 'student', '

Original Sentence: 11/10 would highly recommend walking around campus in mid-afternoon (3-4 ish) while listening to the howl's moving castle main theme (or any studio ghibli soundtrack for that matter),, i did that a few days ago on my way to bel from covered courts, and with that being the accompaniment to the breeze and the not-too-harsh sunlight, it made me forget i had problems and deadlines for at least 5 minutes... ang ganda ng feeling.. thanks joe hisashi
Tokens: ['11/10', 'would', 'highly', 'recommend', 'walking', 'around', 'campus', 'afternoon', 'listening', 'howl', 'move', 'castle', 'main', 'theme', 'studio', 'ghibli', 'soundtrack', 'matter', 'days', 'cover', 'court', 'accompaniment', 'breeze', 'harsh', 'sunlight', 'make', 'forget', 'problem', 'deadline', 'least', 'minutes', 'ganda', 'feeling', 'thanks', 'hisashi']

Original Sentence: Dami niyong oras mag facebook ah. Pahingi naman
Tokens: ['dami', 'niyong', 'oras', 'facebook', 'pahingi', 'naman']

Original Sentence: Re: ADMU

# Bag of Words

In [8]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

pickle.dump(corpus, open('models/corpus.pkl', 'wb'))
dictionary.save('models/dictionary.gensim')

In [9]:
# Analysis of term frequency
vocab = list(dictionary.values()) #list of terms in the dictionary
vocab_tf = [dict(i) for i in corpus]
vocab_tf = pd.DataFrame(vocab_tf).sum(axis=0) #list of term frequencies

test_df = pd.DataFrame()
test_df['Term'] = dictionary.values()
test_df['Frequency'] = vocab_tf
test_df.sort_values(by='Frequency', ascending=False)

,Term,Frequency
62,people,308.0
236,friend,224.0
99,even,223.0
58,love,218.0
67,really,202.0
293,want,201.0
328,make,171.0
85,think,171.0
140,feel,169.0
556,lang,161.0


In [10]:
with open('output/tokens.txt', 'w', encoding='utf-8-sig') as f:
    for word in list(test_df['Term']):
        f.write('{}\n'.format(word))

In [11]:
# Find occurence of a specific term
test_df[test_df['Term'] == 'crush']

,Term,Frequency
529,crush,96.0


# Find topics

In [12]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('models/model5.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.008*"even" + 0.006*"people" + 0.006*"feel" + 0.006*"please"')
(1, '0.015*"friend" + 0.014*"love" + 0.013*"want" + 0.011*"people"')
(2, '0.009*"people" + 0.007*"love" + 0.007*"friend" + 0.006*"think"')
(3, '0.012*"people" + 0.011*"lang" + 0.007*"think" + 0.007*"even"')
(4, '0.007*"post" + 0.006*"really" + 0.006*"hindi" + 0.006*"even"')


# pyLDAvis for visualization

In [13]:
dictionary = gensim.corpora.Dictionary.load('models/dictionary.gensim')
corpus = pickle.load(open('models/corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('models/model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)